In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# 기본 글꼴을 NanumGothic으로 변경
rc("font", family="NanumGothic")

AS1_01_EXAMINEE = pd.read_csv("./AS1_01_EXAMINEE.csv", index_col=0, low_memory=False)
AS1_03_DRSM = pd.read_csv("./AS1_03_DRSM.csv", index_col=0, low_memory=False)
AS1_17_FFQWEIGHT = pd.read_csv("AS1_17_FFQWEIGHT.csv", index_col=0, low_memory=False)


df = pd.DataFrame(index=AS1_01_EXAMINEE.index)

In [2]:
AS1_17_FFQWEIGHT.dropna(inplace=True)

In [3]:
len(AS1_17_FFQWEIGHT)

9704

## 종속변수 처리하기

In [4]:
AS1_06_DISEASE = pd.read_csv("./AS1_06_DISEASE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_07_TREAT = pd.read_csv("./AS1_07_TREAT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_08_DRUG = pd.read_csv("./AS1_08_DRUG.csv", index_col=0, encoding='utf-8', low_memory=False)

disease = AS1_06_DISEASE[["AS1_PDMI", "AS1_PDCH", "AS1_PDCD", "AS1_PDCV"]]
treat = AS1_07_TREAT[["AS1_TRTMI", "AS1_TRTCH", "AS1_TRTCD", "AS1_TRTCV"]]
drug = AS1_08_DRUG[['AS1_DRUGSTKCU']]

dependent = pd.concat([disease, treat, drug], axis=1)

dependent.info()


<class 'pandas.core.frame.DataFrame'>
Index: 10030 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   AS1_PDMI       10030 non-null  int64
 1   AS1_PDCH       10030 non-null  int64
 2   AS1_PDCD       10030 non-null  int64
 3   AS1_PDCV       10030 non-null  int64
 4   AS1_TRTMI      10030 non-null  int64
 5   AS1_TRTCH      10030 non-null  int64
 6   AS1_TRTCD      10030 non-null  int64
 7   AS1_TRTCV      10030 non-null  int64
 8   AS1_DRUGSTKCU  10030 non-null  int64
dtypes: int64(9)
memory usage: 783.6+ KB


In [5]:
dependent.head()

,AS1_PDMI,AS1_PDCH,AS1_PDCD,AS1_PDCV,AS1_TRTMI,AS1_TRTCH,AS1_TRTCD,AS1_TRTCV,AS1_DRUGSTKCU
DIST_ID,,,,,,,,,
NIH23B1298125,1,1,1,1,1,1,1,1,77777
NIH23B1159376,1,1,1,1,99999,99999,99999,99999,99999
NIH23B1785393,1,1,1,1,99999,99999,99999,99999,77777
NIH23B1463054,1,1,1,1,1,1,1,1,77777
NIH23B1751168,1,1,1,1,99999,99999,99999,99999,99999


In [6]:
# 모든 값이 77777 또는 99999이면 drop
mask = ~dependent.isin([77777, 99999]).all(axis=1)
dependent = dependent[mask]

In [7]:
# 3명 drop함. 
dependent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10027 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   AS1_PDMI       10027 non-null  int64
 1   AS1_PDCH       10027 non-null  int64
 2   AS1_PDCD       10027 non-null  int64
 3   AS1_PDCV       10027 non-null  int64
 4   AS1_TRTMI      10027 non-null  int64
 5   AS1_TRTCH      10027 non-null  int64
 6   AS1_TRTCD      10027 non-null  int64
 7   AS1_TRTCV      10027 non-null  int64
 8   AS1_DRUGSTKCU  10027 non-null  int64
dtypes: int64(9)
memory usage: 783.4+ KB


In [8]:
def create_cvd_column(df):
    # 해당 열들의 값이 2인지 확인
    conditions = (
        (df['AS1_PDMI'] == 2) |
        (df['AS1_PDCH'] == 2) |
        (df['AS1_PDCD'] == 2) |
        (df['AS1_PDCV'] == 2) |
        (df['AS1_TRTMI'] == 2) |
        (df['AS1_TRTCH'] == 2) |
        (df['AS1_TRTCD'] == 2) |
        (df['AS1_TRTCV'] == 2) |
        (df['AS1_DRUGSTKCU'] == 2)
    )
    
    # 조건에 맞으면 1, 아니면 0 할당
    df['CVD'] = conditions.astype(int)
    
    return df

In [9]:
df = create_cvd_column(dependent.copy())
print(df["CVD"].value_counts())

print(df.info())
df.head()

CVD
0    9726
1     301
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 10027 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   AS1_PDMI       10027 non-null  int64
 1   AS1_PDCH       10027 non-null  int64
 2   AS1_PDCD       10027 non-null  int64
 3   AS1_PDCV       10027 non-null  int64
 4   AS1_TRTMI      10027 non-null  int64
 5   AS1_TRTCH      10027 non-null  int64
 6   AS1_TRTCD      10027 non-null  int64
 7   AS1_TRTCV      10027 non-null  int64
 8   AS1_DRUGSTKCU  10027 non-null  int64
 9   CVD            10027 non-null  int32
dtypes: int32(1), int64(9)
memory usage: 822.5+ KB
None


,AS1_PDMI,AS1_PDCH,AS1_PDCD,AS1_PDCV,AS1_TRTMI,AS1_TRTCH,AS1_TRTCD,AS1_TRTCV,AS1_DRUGSTKCU,CVD
DIST_ID,,,,,,,,,,
NIH23B1298125,1,1,1,1,1,1,1,1,77777,0
NIH23B1159376,1,1,1,1,99999,99999,99999,99999,99999,0
NIH23B1785393,1,1,1,1,99999,99999,99999,99999,77777,0
NIH23B1463054,1,1,1,1,1,1,1,1,77777,0
NIH23B1751168,1,1,1,1,99999,99999,99999,99999,99999,0


In [10]:
print(dependent.info())
dependent.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10027 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   AS1_PDMI       10027 non-null  int64
 1   AS1_PDCH       10027 non-null  int64
 2   AS1_PDCD       10027 non-null  int64
 3   AS1_PDCV       10027 non-null  int64
 4   AS1_TRTMI      10027 non-null  int64
 5   AS1_TRTCH      10027 non-null  int64
 6   AS1_TRTCD      10027 non-null  int64
 7   AS1_TRTCV      10027 non-null  int64
 8   AS1_DRUGSTKCU  10027 non-null  int64
dtypes: int64(9)
memory usage: 783.4+ KB
None


,AS1_PDMI,AS1_PDCH,AS1_PDCD,AS1_PDCV,AS1_TRTMI,AS1_TRTCH,AS1_TRTCD,AS1_TRTCV,AS1_DRUGSTKCU
DIST_ID,,,,,,,,,
NIH23B1298125,1,1,1,1,1,1,1,1,77777
NIH23B1159376,1,1,1,1,99999,99999,99999,99999,99999
NIH23B1785393,1,1,1,1,99999,99999,99999,99999,77777
NIH23B1463054,1,1,1,1,1,1,1,1,77777
NIH23B1751168,1,1,1,1,99999,99999,99999,99999,99999


In [11]:
dependent = df # CVD 변수 추가

In [12]:
dependent.to_csv("./dropped_dependent.csv")

## 식이지수 처리할 준비하기

### AS1_TOTALC 변수 결측값 처리

In [13]:
drsm = AS1_03_DRSM.loc[dependent.index]
drsm = drsm.replace(77777, 0)
drsm = drsm.replace(99999, np.nan)

In [14]:
missing_totalc = drsm[(drsm['AS1_DRINK'] == 3.0) & (drsm['AS1_TOTALC'].isnull())]
len(missing_totalc)

214

In [15]:
missing_totalc

,AS1_DRINK,AS1_DRDUA,AS1_DRQMO,AS1_TAK,AS1_TAKFQA,AS1_TAKAMA,AS1_TAKGS,AS1_BEER,AS1_BEERFQA,AS1_BEERAMA,...,AS1_PSTSMYR,AS1_PSTSMMO,AS1_PSTSMAM,AS1_PACKYR,AS1_PSM,AS1_PSMH,AS1_PSMHFQA,AS1_PSMHMIN,AS1_PSMOFQA,AS1_PSMOMIN
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1240373,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,20.0,2.0,1.0,3.0,360.0,0.0,0.0
NIH23B1498753,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,0.0,6.0,0.6,1.0,0.0,0.0,0.0,0.0,0.0
NIH23B1054423,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,7.5,1.0,0.0,0.0,0.0,0.0,0.0
NIH23B1895877,3.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,1.0,10.0,0.0,0.0
NIH23B1357709,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,20.0,0.0,10.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NIH23B1389490,3.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
NIH23B1460564,3.0,1.0,0.0,2.0,NaN,NaN,NaN,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,NaN,0.0,0.0,1.0,5.0
NIH23B1083518,3.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2.0,1.0,3.0,30.0,0.0,0.0


In [16]:
# 주종 리스트
alcohols = ['AS1_TAK', 'AS1_BEER', 'AS1_RICE', 'AS1_WINE', 'AS1_SOJU', 'AS1_HLIQ']

for alcohol in alcohols:
   # FQA 열 결측치 처리
   fqa_col = alcohol + 'FQA'
   fqa_mean = drsm.loc[drsm[alcohol] == 2, fqa_col].mean()
   drsm[fqa_col] = drsm[fqa_col].fillna(round(fqa_mean))

   # AMA 열 결측치 처리
   ama_col = alcohol + 'AMA'
   ama_mean = drsm.loc[drsm[alcohol] == 2, ama_col].mean()
   drsm[ama_col] = drsm[ama_col].fillna(ama_mean)

   gs_col = alcohol + 'GS'
   gs_freq = drsm.loc[drsm[alcohol] == 2, gs_col].mode()[0]
   

In [17]:
drsm['AS1_TOTALC'].value_counts()

AS1_TOTALC
0.00     5247
20.26     163
11.58     119
8.68      111
40.52     104
         ... 
38.20       1
19.42       1
68.07       1
69.75       1
2.44        1
Name: count, Length: 1184, dtype: int64

In [18]:
# 알코올 도수 딕셔너리
alc_content = {
   'AS1_TAK': 0.06,
   'AS1_BEER': 0.045,
   'AS1_RICE': 0.15,
   'AS1_WINE': 0.13,
   'AS1_SOJU': 0.22,
   'AS1_HLIQ': 0.40
}

# AS1_TOTALC 칼럼 결측치 대치
for idx, row in missing_totalc.iterrows():
    total_alcohol = 0
    for alcohol in alc_content.keys():
        freq_col = alcohol + 'FQA'
        amount_col = alcohol + 'AMA'
        
        # 해당 주종의 알코올 섭취량 계산
        alcohol_amount = alc_content[alcohol] * row[freq_col] * row[amount_col] * 0.7893
        
        # 총 알코올 섭취량에 합산
        total_alcohol += alcohol_amount
    
    # AS1_TOTALC 결측치 대치
    drsm.at[idx, 'AS1_TOTALC'] = total_alcohol

In [19]:
drsm.head()

,AS1_DRINK,AS1_DRDUA,AS1_DRQMO,AS1_TAK,AS1_TAKFQA,AS1_TAKAMA,AS1_TAKGS,AS1_BEER,AS1_BEERFQA,AS1_BEERAMA,...,AS1_PSTSMYR,AS1_PSTSMMO,AS1_PSTSMAM,AS1_PACKYR,AS1_PSM,AS1_PSMH,AS1_PSMHFQA,AS1_PSMHMIN,AS1_PSMOFQA,AS1_PSMOMIN
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,7.0,0.0,10.0,3.5,1.0,0.0,0.0,0.0,0.0,0.0
NIH23B1159376,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
NIH23B1785393,3.0,4.0,0.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,...,0.0,0.0,0.0,8.0,2.0,NaN,2.0,10.0,3.0,20.0
NIH23B1463054,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,3.0,5.0,2.0,NaN
NIH23B1751168,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0


### 완료

In [20]:
sex_age = AS1_01_EXAMINEE[["AS1_SEX", "AS1_AGE"]]
drink = drsm[["AS1_DRINK", "AS1_TOTALC"]]

df = pd.concat([sex_age, drink, AS1_17_FFQWEIGHT], axis=1, join='inner') # inner join 겹치는 인덱스만 합쳐야함 

df.head()

,AS1_SEX,AS1_AGE,AS1_DRINK,AS1_TOTALC,AS1_W001,AS1_W002,AS1_W003,AS1_W004,AS1_W005,AS1_W006,...,AS1_W094,AS1_W095,AS1_W096,AS1_W097,AS1_W098,AS1_W099,AS1_W100,AS1_W101,AS1_W102,AS1_W103
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,3.0,23.15,220.0,0.0,440.0,1.0,10.0,64.285714,...,0.0,4.0,0.0,16.666667,0.000000,0.000000,0.000000,120.0,0.0,32.142857
NIH23B1159376,2,44,1.0,0.00,440.0,0.0,0.0,0.0,4.0,10.000000,...,0.0,0.0,0.0,16.666667,0.578571,1.071429,0.857143,0.0,0.0,0.000000
NIH23B1785393,1,47,3.0,23.57,660.0,0.0,0.0,0.0,10.0,150.000000,...,0.0,0.0,0.0,0.000000,2.700000,5.000000,0.000000,0.0,0.0,0.000000
NIH23B1463054,2,43,3.0,0.29,660.0,0.0,0.0,0.0,10.0,150.000000,...,0.0,10.0,0.0,16.666667,2.700000,5.000000,0.000000,0.0,0.0,0.000000
NIH23B1751168,1,61,3.0,43.41,990.0,0.0,0.0,0.0,0.0,10.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [21]:
df.to_csv("./ready_to_create_NQ.csv")

## 테스트

In [27]:
print(f"""
dependent:          {len(dependent)}
sex_age:            {len(sex_age)}
drink:              {len(drink)}
AS1_17_FFQWEIGHT:   {len(AS1_17_FFQWEIGHT)}
""")


dependent:          10027
sex_age:            10030
drink:              10027
AS1_17_FFQWEIGHT:   9704



In [24]:
df = pd.read_csv("./ready_to_create_NQ.csv")

In [25]:
len(df)

9701